In [146]:
import numpy as np
import math 
import random
from numpy import linalg 
from numpy import real
from scipy import linalg, matrix
from sympy import Matrix
from fractions import Fraction 
import sympy as sy

A={0:(0,1), 1:(2,3), 2:(0,2), 3:(0,3), 4:(1,2), 5:(1,3)}

kex=3
kin=1

def distance(X1,X2):
	Sum=0.0
	for i in range(len(X1)):
		Sum=Sum+(X1[i]-X2[i])*(X1[i]-X2[i])
	return math.sqrt(Sum/len(X1))

### Sorting things ### 
def getKey(item):
    return item[0]


### Generating all sets of states, 00, 01, 10, 11  ### 
def OrderSets():
#	List=[0]*6
	Allset=[]
	List=[]
#	Allset.append(List)
#	print Allset
	NewSet=[]
	Set(Allset,List,0)
	for i in range(len(Allset)):
#		print list(reversed(Allset[i]))
		NewSet.append(Allset[i])
#		print Allset[i]
#		NewSet.append(list(reversed(Allset[i])))
#	print list(reversed(NewSet))
	return NewSet
#	return NewSet



## Generating all sets of states 00, 10, 01, 11 ##
def AllSets():
	Allset=[]
	List=[]
	NewSet=[]
	Set(Allset,List,0)
	for i in range(len(Allset)):
		NewSet.append(list(reversed(Allset[i])))
	return NewSet

def Set(Allset,List,index):
#	print index
	if len(List)==6:
		Allset.append(List)
	else:
		for i in range(2):
			Set(Allset,List+[i],index+1)
## Indexing the states ## 

## Indexing the states ## 
def state(N1):
	N=list(reversed(N1))
	L=2**N[0]
	for i in range(1,len(N)):
		L=L+N[i]*(2**((i)*N[i]))
	return L-1



## Measuring transition probability ##
def stay(N1,k):
    L1=state(N1)
    prob=0
    if k[0]==kin:
        prob=prob+1
    if k[1]==kin:
        prob=prob+1
    if k[2]==kex:
        prob=prob+1
    if k[3]==kex:
        prob=prob+1
    if prob==0:
        prob=0
    return (L1,sy.Rational(Fraction(prob,4)))

def Tran(N1,N2,k):
    prob=0
    L1=state(N1)
    L2=state(N2)

    index=[]
    
    for i in range(len(N1)):
        if N1[i]!=N2[i]:
            prob=0
            if N1[i]==0:
                if i in range(2,6):
                    (x,y)=A[i]
                    if k[y] < kex:
                        prob=prob+sy.Rational(Fraction(1,4*(3-k[y])))
                    if k[x] < kin:
                        prob=prob+sy.Rational(Fraction(1,4*(3-k[x])))

                if i == 0:
                    (x,y)=A[i]                    
                    if k[y] < kin:
                        prob=prob+sy.Rational(Fraction(1,4*(3-k[y])))
                    if k[x] < kin:
                        prob=prob+sy.Rational(Fraction(1,4*(3-k[x])))
                                                
                if i == 1:
                    (x,y)=A[i]                    
                    if k[y] < kex:
                        prob=prob+sy.Rational(Fraction(1,4*(3-k[y])))
                    if k[x] < kex:
                        prob=prob+sy.Rational(Fraction(1,4*(3-k[x])))
                    
                
            if N1[i]!=0:
                if i in range(2,6):
                    (x,y)=A[i]
                    if k[y] > kex:
                        prob=prob+sy.Rational(Fraction(1,4*(k[y])))
                    if k[x] > kin:
                        prob=prob+sy.Rational(Fraction(1,4*(k[x])))

                if i == 0:
                    (x,y)=A[i]                    
                    if k[y] > kin:
                        prob=prob+sy.Rational(Fraction(1,4*(k[y])))
                    if k[x] > kin:
                        prob=prob+sy.Rational(Fraction(1,4*(k[x])))
                                                
                if i == 1:
                    (x,y)=A[i]                    
                    if k[y] > kex:
                        prob=prob+sy.Rational(Fraction(1,4*(k[y])))
                    if k[x] > kex:
                        prob=prob+sy.Rational(Fraction(1,4*(k[x])))
                                
    #print (L2,prob)
    return (L2,prob)




## returning W matrix ## 

def WMatrix():
    W=np.zeros((64, 64))
    W=W.astype('object')
    for i in range(64):
        for j in range(64):
            W[i,j]=sy.Rational('0')
    New=OrderSets()
    for i in New:
        Nb(i, W)
    return W 


## Printing W Matrix in Latex Format ## 
#fracW=np.zeros((64,64))
#fracW=fracW.astype('object')





## Measuring the row related state N1 in W matrix ## 
def Nb(N1, W):
    L1=state(N1)
    Neig=[]
    k=[0]*4

    for i in range(len(N1)):
        Neig.append(N1[:i]+[1-N1[i]]+N1[i+1:])

    k[0]=N1[0]+N1[2]+N1[3]
    k[1]=N1[0]+N1[4]+N1[5]
    k[2]=N1[2]+N1[4]+N1[1]
    k[3]=N1[3]+N1[5]+N1[1]

    Grand=[]
    Grand.append(stay(N1,k))
    for i in Neig:
        Z=Tran(N1,i,k)
        Grand.append(Z)
    Grand1=sorted(Grand,key=getKey)
    for i in range(len(Grand1)-1):
        (x,y)=Grand1[i]
        W[L1][x]=y
    (x,y)=Grand1[len(Grand1)-1]
    W[L1][x]=y


W= WMatrix()
WT=W.T

In [147]:
symStates={0:[16],1:[17,18,20,24],2:[19,28],3:[21,26],4:[22,25],5:[23,27,29,30],6:[31],7:[48],8:[32+17,32+18,32+20,32+24],9:[32+19,32+28],10:[32+21,32+26],11:[32+22,32+25],12:[32+23,32+27,32+29,32+30],13:[32+31]}
SurvList=[i for i in range(16,32)]+[i for i in range(48,64)]
def returnKey(value,Dict):
    for i in Dict:
        for j in Dict[i]:
            if value == j:
                return i 

In [151]:
## Providing list of Eq for surviving states, feeding in Mathematica ## 
#s=""
for i in range(14):
    coefList=[0]*14  
    coefList[i]=-1
    s=symStates[i]
    state=s[0]
    List=[]
    for j in SurvList:
        contState=returnKey(j,symStates)
        if WT[state,j] > 0:
            coefList[contState]=coefList[contState]+WT[state,j]
    coef=[]
    s=""
    for z in range(14):
        if coefList[z]!=0:
            coef.append((z+1,coefList[z]))
    
    for L in range(len(coef)):
        (x,y)=coef[L]
        s=s+"DZ[["+str(i+1)+","+str(x)+"]]="+str(y)+";\n"
    print s
#    print (i+1,coef)

DZ[[1,1]]=-1;

DZ[[2,1]]=5/24;
DZ[[2,2]]=-3/4;
DZ[[2,3]]=1/8;
DZ[[2,9]]=1/8;

DZ[[3,2]]=1/4;
DZ[[3,3]]=-1;
DZ[[3,10]]=1/12;

DZ[[4,2]]=2/3;
DZ[[4,4]]=-1/4;
DZ[[4,6]]=1/4;
DZ[[4,11]]=1/4;

DZ[[5,2]]=5/12;
DZ[[5,5]]=-1/2;
DZ[[5,6]]=1/4;
DZ[[5,12]]=1/4;

DZ[[6,3]]=1/3;
DZ[[6,4]]=1/8;
DZ[[6,5]]=1/4;
DZ[[6,6]]=-1/2;
DZ[[6,7]]=1/8;
DZ[[6,13]]=5/24;

DZ[[7,6]]=1;
DZ[[7,7]]=-1/2;
DZ[[7,14]]=1/6;

DZ[[8,1]]=1/6;
DZ[[8,8]]=-1/2;
DZ[[8,9]]=1/2;

DZ[[9,2]]=1/12;
DZ[[9,8]]=1/8;
DZ[[9,9]]=-3/4;
DZ[[9,10]]=1/12;
DZ[[9,11]]=1/8;
DZ[[9,12]]=1/8;

DZ[[10,3]]=1/12;
DZ[[10,9]]=1/4;
DZ[[10,10]]=-3/4;
DZ[[10,13]]=1/4;

DZ[[11,9]]=1/2;
DZ[[11,11]]=-3/4;
DZ[[11,13]]=1/6;

DZ[[12,9]]=1/4;
DZ[[12,12]]=-1;
DZ[[12,13]]=1/6;

DZ[[13,10]]=1/4;
DZ[[13,11]]=1/8;
DZ[[13,12]]=1/4;
DZ[[13,13]]=-3/4;
DZ[[13,14]]=1/12;

DZ[[14,13]]=1;
DZ[[14,14]]=-1/2;



In [152]:
sState=[sy.Rational(0)]*14

In [153]:
## Input from Mathematica ## 
sState[3]=sy.Rational()

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]